In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
# 2. xavier_init 추가
def xavier_initializer(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [11]:
# parameter 설정
learning_rate = 0.001
training_epochs = 25
batch_size = 100
display_step=1 

In [4]:
X = tf.placeholder('float', [None, 784]) # mnist data image of shape 28 * 28 = 784
Y = tf.placeholder('float', [None, 10]) # 0-9 digits recognition = > 10 classes

In [7]:
# 1. set model weights
# W1 = tf.Variable(tf.random_normal([784, 256]))
# W2 = tf.Variable(tf.random_normal([256, 256]))
# W3 = tf.Variable(tf.random_normal([256, 10]))

# 2. xavier init 으로 수정

# W1 = tf.get_variable("W1", shape=[784, 256], initializer=xavier_init(784, 256))
# W2 = tf.get_variable("W2", shape=[256, 256], initializer=xavier_init(256, 256))
# W3 = tf.get_variable("W3", shape=[256,  10], initializer=xavier_init(256,  10))

# B1 = tf.Variable(tf.random_normal([256]))
# B2 = tf.Variable(tf.random_normal([256]))
# B3 = tf.Variable(tf.random_normal([10]))

# 3. 5-layer deep neural network #####################################################
dropout_rate = tf.placeholder("float")
W__1 = tf.get_variable("W__1", shape=[784, 256], initializer=xavier_initializer(784, 256))
W__2 = tf.get_variable("W__2", shape=[256, 256], initializer=xavier_initializer(256, 256))
W__3 = tf.get_variable("W__3", shape=[256, 256], initializer=xavier_initializer(256, 256))
W__4 = tf.get_variable("W__4", shape=[256, 256], initializer=xavier_initializer(256, 256))
W__5 = tf.get_variable("W__5", shape=[256, 10], initializer=xavier_initializer(256, 10))

B1 = tf.Variable(tf.random_normal([256]))
B2 = tf.Variable(tf.random_normal([256]))
B3 = tf.Variable(tf.random_normal([256]))
B4 = tf.Variable(tf.random_normal([256]))
B5 = tf.Variable(tf.random_normal([10]))

####################################################################################

# construct model
# L1 = tf.nn.relu(tf.add(tf.matmul(X,W1), B1))
# L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), B2))
# hypothesis = tf.add(tf.matmul(L2, W3), B3)

# Construct model
_L1 = tf.nn.relu(tf.add(tf.matmul(X,W__1),B1))
L1 = tf.nn.dropout(_L1, dropout_rate)
_L2 = tf.nn.relu(tf.add(tf.matmul(L1, W__2),B2)) # Hidden layer with ReLU activation
L2 = tf.nn.dropout(_L2, dropout_rate)
_L3 = tf.nn.relu(tf.add(tf.matmul(L2, W__3),B3)) # Hidden layer with ReLU activation
L3 = tf.nn.dropout(_L3, dropout_rate)
_L4 = tf.nn.relu(tf.add(tf.matmul(L3, W__4),B4)) # Hidden layer with ReLU activation
L4 = tf.nn.dropout(_L4, dropout_rate)

hypothesis = tf.add(tf.matmul(L4, W__5), B5) # No need to use softmax here

In [8]:
# cost(softmax loss)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y)) 

# gradient descent
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [14]:
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        # 나누어 떨어지지 않을경우 이미지 일부는 사용 않함
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#             _, c = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys})
            _, c = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys, dropout_rate:0.7})
            
            # cost 누적
            avg_cost += c / total_batch
        
        if(epoch+1) % display_step == 0:
            print("Epoch : ", "%04d" % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    
    print("Optimization Finished!")

    # Testmodel
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y,1))

    # calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy : ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y:mnist.test.labels, dropout_rate : 1}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch :  0001 cost= 0.586970997
Epoch :  0002 cost= 0.220020569
Epoch :  0003 cost= 0.171463429
Epoch :  0004 cost= 0.141989639
Epoch :  0005 cost= 0.120800585
Epoch :  0006 cost= 0.105133912
Epoch :  0007 cost= 0.095749959
Epoch :  0008 cost= 0.090603283
Epoch :  0009 cost= 0.085706684
Epoch :  0010 cost= 0.076853301
Epoch :  0011 cost= 0.075029351
Epoch :  0012 cost= 0.070623538
Epoch :  0013 cost= 0.066295225
Epoch :  0014 cost= 0.061612324
Epoch :  0015 cost= 0.058700139
Epoch :  0016 cost= 0.057191131
Epoch :  0017 cost= 0.056565330
Epoch :  0018 cost= 0.053543506
Epoch :  0019 cost= 0.051378120
Epoch :  0020 cost= 0.051510622
Epoch :  0021 cost= 0.047839428
Epoch :  0022 cost= 0.047274687
Epoch :  0023 cost= 0.045261268
Epoch :  0024 cost= 0.046026430
Epoch :  0025 cost= 0.042600495
Optimization Finished!
Accuracy :  0.9824
